# [Predicting the price of Bitcoin with multivariate Pytorch LSTMs - revise](https://charlieoneill.medium.com/predicting-the-price-of-bitcoin-with-multivariate-pytorch-lstms-695bc294130)

In [6]:
'''Train with PyTorch.'''
# PyTorch
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.data.dataset import random_split
import torch.utils.data as data

# SciKit
from sklearn.preprocessing import MinMaxScaler, StandardScaler

# Python
import pandas as pd
import numpy as np
import os
import time

In [2]:
df = pd.read_csv('./station/466920taipei_train.csv')
df = df[["觀測時間(hour)", "氣溫(℃)", "相對溼度(%)", "露點溫度(℃)", "測站氣壓(hPa)"]]
df.head()

,觀測時間(hour),氣溫(℃),相對溼度(%),露點溫度(℃),測站氣壓(hPa)
0,2010/1/1 00:00,13.7,69.0,8.2,1021.8
1,2010/1/1 01:00,13.8,69.0,8.3,1021.4
2,2010/1/1 02:00,13.9,69.0,8.4,1020.4
3,2010/1/1 03:00,13.9,69.0,8.4,1019.9
4,2010/1/1 04:00,14.0,69.0,8.5,1019.6


In [8]:
features = df[["相對溼度(%)", "露點溫度(℃)", "測站氣壓(hPa)"]]
targets = df[["氣溫(℃)"]].values

features.shape, targets.shape

((96360, 3), (96360, 1))

In [9]:
mm = MinMaxScaler()
ss = StandardScaler()

features_trans = ss.fit_transform(features)
targets_trans = mm.fit_transform(targets.reshape(-1, 1))

In [10]:
def split_sequences(input_sequences, output_sequence, n_steps_in, n_steps_out):
    
    features, targets = list(), list()
    
    for i in range(len(input_sequences)):
        
        # find the end of the input, output sequence
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out - 1

        # check if we are beyond the dataset
        if out_end_ix > len(input_sequences): break

        # gather input and output of the pattern
        seq_x, seq_y = input_sequences[i:end_ix], output_sequence[end_ix-1:out_end_ix, -1]
        features.append(seq_x), targets.append(seq_y)

    return np.array(features), np.array(targets)

features_ss, targets_mm = split_sequences(features_trans, targets_trans, 100, 50)  # 100跟50不知道是什麼意思
print(features_ss.shape, targets_mm.shape)

(96212, 100, 3) (96212, 50)


In [15]:
assert targets_mm[0].all() == targets_trans[99:149].squeeze(1).all()

targets_mm[0]

array([0.3591954 , 0.35344828, 0.33333333, 0.32183908, 0.31034483,
       0.29022989, 0.28448276, 0.29310345, 0.29310345, 0.31609195,
       0.32758621, 0.30747126, 0.30172414, 0.29310345, 0.28448276,
       0.28448276, 0.27298851, 0.27011494, 0.27298851, 0.27586207,
       0.27873563, 0.29022989, 0.29597701, 0.29885057, 0.29885057,
       0.29310345, 0.29597701, 0.28735632, 0.29022989, 0.29597701,
       0.29310345, 0.29022989, 0.29310345, 0.30172414, 0.3045977 ,
       0.3045977 , 0.3045977 , 0.30172414, 0.29310345, 0.28448276,
       0.2816092 , 0.2816092 , 0.2816092 , 0.28735632, 0.27586207,
       0.27298851, 0.27011494, 0.27011494, 0.26724138, 0.27011494])

In [16]:
targets_trans[99:149].squeeze(1)

array([0.3591954 , 0.35344828, 0.33333333, 0.32183908, 0.31034483,
       0.29022989, 0.28448276, 0.29310345, 0.29310345, 0.31609195,
       0.32758621, 0.30747126, 0.30172414, 0.29310345, 0.28448276,
       0.28448276, 0.27298851, 0.27011494, 0.27298851, 0.27586207,
       0.27873563, 0.29022989, 0.29597701, 0.29885057, 0.29885057,
       0.29310345, 0.29597701, 0.28735632, 0.29022989, 0.29597701,
       0.29310345, 0.29022989, 0.29310345, 0.30172414, 0.3045977 ,
       0.3045977 , 0.3045977 , 0.30172414, 0.29310345, 0.28448276,
       0.2816092 , 0.2816092 , 0.2816092 , 0.28735632, 0.27586207,
       0.27298851, 0.27011494, 0.27011494, 0.26724138, 0.27011494])

In [18]:
total_samples = len(features)
train_test_split = round(0.7 * total_samples)

features_train = features_ss[:-150]
features_test = features_ss[-150:]

targets_train = targets_mm[:-150]
targets_test = targets_mm[-150:]

print("Training Shape:", features_train.shape, targets_train.shape)
print("Testing Shape:", features_test.shape, targets_test.shape) 

Training Shape: (96062, 100, 3) (96062, 50)
Testing Shape: (150, 100, 3) (150, 50)


In [19]:
from torch.autograd import Variable  # 导入torch中Variable模块

# convert to pytorch tensors
features_train_tensors = Variable(torch.Tensor(features_train))
features_test_tensors = Variable(torch.Tensor(features_test))

targets_train_tensors = Variable(torch.Tensor(targets_train))
targets_test_tensors = Variable(torch.Tensor(targets_test))

In [20]:
# reshaping to rows, timestamps, features
features_train_tensors_final = torch.reshape(features_train_tensors, 
                                      (features_train_tensors.shape[0], 100, 
                                       features_train_tensors.shape[2]))
features_test_tensors_final = torch.reshape(features_test_tensors,  
                                     (features_test_tensors.shape[0], 100, 
                                      features_test_tensors.shape[2])) 

print("Training Shape:", features_train_tensors_final.shape, targets_train_tensors.shape)
print("Testing Shape:", features_test_tensors_final.shape, targets_test_tensors.shape) 

Training Shape: torch.Size([96062, 100, 3]) torch.Size([96062, 50])
Testing Shape: torch.Size([150, 100, 3]) torch.Size([150, 50])
